# PyTorch 从零实现知识蒸馏

使用 Qwen2.5-0.5B 作为教师模型，从零实现 ~123M 参数的 Decoder-Only Transformer 学生模型，
在 Wikipedia 中文子集上进行在线知识蒸馏训练。

**运行环境**: Google Colab T4 GPU (15GB VRAM)

**模块结构**:
- `src/config.py` — 配置数据类
- `src/model.py` — 模型架构（RMSNorm, RoPE, GQA, SwiGLU）
- `src/data.py` — 数据加载与预处理
- `src/trainer.py` — 蒸馏训练循环
- `src/generate.py` — 文本生成推理

## Cell 1: 环境检查 & 依赖安装

In [ ]:
# 安装依赖（Colab 环境）
# !pip install torch transformers datasets matplotlib

import sys
import torch

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n使用设备: {DEVICE}")

## Cell 2: 挂载 Google Drive（可选）

In [ ]:
# 挂载 Google Drive 用于持久化检查点（可选）
# 如果在本地运行，可跳过此 Cell

USE_DRIVE = False  # 设置为 True 启用 Google Drive

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    CHECKPOINT_DIR = '/content/drive/MyDrive/distillation_checkpoints/'
else:
    CHECKPOINT_DIR = 'checkpoints/'

import os
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
print(f"检查点目录: {CHECKPOINT_DIR}")

## Cell 3: 导入模块 & 配置

In [ ]:
# 如果在 Colab 中，需要先克隆项目或上传 src/ 目录
# !git clone <repo-url> && cd LLM

import sys
sys.path.insert(0, '..')  # 确保可以导入 src 模块

from src.config import ModelConfig, TrainingConfig
from src.model import StudentModel
from src.data import load_tokenizer, create_dataloaders
from src.trainer import DistillationTrainer, load_teacher_model
from src.generate import TextGenerator, load_trained_model

# 模型配置
model_config = ModelConfig()
print(f"模型配置: {model_config}")
print(f"预期参数量: ~{151665 * 512 + 3802112 * 12 + 512:,}")

# 训练配置
training_config = TrainingConfig(checkpoint_dir=CHECKPOINT_DIR)
print(f"\n训练配置: {training_config}")

## Cell 4: 加载 Tokenizer & 构建数据集

In [ ]:
# 加载 Tokenizer
tokenizer = load_tokenizer()
print(f"Tokenizer vocab_size: {tokenizer.vocab_size}")
print(f"pad_token: {tokenizer.pad_token}")

# 构建 DataLoader（取 5000 条文章，约 50MB 文本）
train_loader, val_loader = create_dataloaders(
    tokenizer=tokenizer,
    config=training_config,
    model_config=model_config,
    max_samples=5000,
)

print(f"\n训练集 batch 数: {len(train_loader)}")
print(f"验证集 batch 数: {len(val_loader)}")

# 验证一个 batch
sample_batch = next(iter(train_loader))
print(f"\n样本 batch:")
print(f"  input_ids shape: {sample_batch['input_ids'].shape}")
print(f"  labels shape: {sample_batch['labels'].shape}")

## Cell 5: 构建学生模型

In [ ]:
# 构建学生模型
student_model = StudentModel(model_config)
param_count = student_model.count_parameters()
print(f"学生模型参数量: {param_count:,} ({param_count / 1e6:.1f}M)")
print(f"\n模型结构:")
print(student_model)

# 验证前向传播
with torch.no_grad():
    test_input = torch.randint(0, model_config.vocab_size, (2, 32))
    test_output = student_model(test_input)
    print(f"\n前向传播验证:")
    print(f"  输入: {test_input.shape}")
    print(f"  输出: {test_output.shape}")
    assert test_output.shape == (2, 32, model_config.vocab_size)
    print("  ✓ 输出形状正确")

# 显存状态
if torch.cuda.is_available():
    student_model = student_model.to(DEVICE)
    print(f"\n显存占用:")
    print(f"  已分配: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"  已缓存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

## Cell 6: 加载教师模型

In [ ]:
# 加载教师模型（Qwen2.5-0.5B, FP16）
teacher_model = load_teacher_model(device=DEVICE)
teacher_params = sum(p.numel() for p in teacher_model.parameters())
print(f"教师模型参数量: {teacher_params:,} ({teacher_params / 1e6:.1f}M)")

# 显存状态
if torch.cuda.is_available():
    print(f"\n显存占用（学生 + 教师）:")
    print(f"  已分配: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"  已缓存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    free_mem = torch.cuda.get_device_properties(0).total_mem - torch.cuda.memory_allocated()
    print(f"  剩余显存: {free_mem / 1024**3:.2f} GB")

## Cell 7: 执行蒸馏训练

In [ ]:
# 创建蒸馏训练器
trainer = DistillationTrainer(
    student_model=student_model,
    teacher_model=teacher_model,
    train_loader=train_loader,
    val_loader=val_loader,
    config=training_config,
    device=DEVICE,
)

# 执行训练
print("开始蒸馏训练...")
history = trainer.train()
print(f"\n训练完成！共 {len(history['train_loss'])} 步")

## Cell 8: 绘制 Loss 曲线

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 训练 Loss 曲线（使用滑动平均平滑）
train_loss = history["train_loss"]
window = min(50, len(train_loss) // 5 + 1)
if len(train_loss) > window:
    smoothed = [sum(train_loss[max(0,i-window):i+1]) / len(train_loss[max(0,i-window):i+1])
                for i in range(len(train_loss))]
else:
    smoothed = train_loss

axes[0].plot(train_loss, alpha=0.3, label="原始")
axes[0].plot(smoothed, label=f"滑动平均 (window={window})")
axes[0].set_xlabel("训练步数")
axes[0].set_ylabel("Loss")
axes[0].set_title("训练 Loss 曲线")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 验证 Loss & PPL
if history["val_loss"]:
    ax2 = axes[1]
    ax2.plot(history["val_loss"], "o-", color="orange", label="验证 Loss")
    ax2.set_xlabel("评估次数")
    ax2.set_ylabel("Loss")
    ax2.set_title("验证集指标")
    ax2_twin = ax2.twinx()
    ax2_twin.plot(history["val_ppl"], "s--", color="green", label="PPL")
    ax2_twin.set_ylabel("Perplexity")
    ax2.legend(loc="upper left")
    ax2_twin.legend(loc="upper right")
    ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("training_curves.png", dpi=150)
plt.show()
print("训练曲线已保存至 training_curves.png")

## Cell 9: 文本生成演示

In [ ]:
import os

# 加载最佳模型
best_ckpt = os.path.join(CHECKPOINT_DIR, "best.pt")
if not os.path.exists(best_ckpt):
    best_ckpt = os.path.join(CHECKPOINT_DIR, "final.pt")

gen_model = load_trained_model(best_ckpt, model_config, DEVICE)
generator = TextGenerator(model=gen_model, tokenizer=tokenizer, device=DEVICE)

# 测试提示词
prompts = [
    "中国的首都是",
    "人工智能的发展",
    "数学是研究",
]

strategies = ["greedy", "top_k", "top_p"]

for prompt in prompts:
    print(f"\n{'='*60}")
    print(f"提示词: {prompt}")
    print(f"{'='*60}")
    for strategy in strategies:
        result = generator.generate(
            prompt,
            max_new_tokens=100,
            strategy=strategy,
            temperature=0.8,
            top_k=50,
            top_p=0.9,
        )
        print(f"\n[{strategy}]:")
        print(result)

## Cell 10: 保存最终模型

In [ ]:
import os

# 保存最终模型和配置
final_save_dir = os.path.join(CHECKPOINT_DIR, "final_model")
os.makedirs(final_save_dir, exist_ok=True)

# 保存模型权重
torch.save(
    {
        "model_state_dict": gen_model.state_dict(),
        "model_config": model_config,
        "training_config": training_config,
    },
    os.path.join(final_save_dir, "model.pt"),
)

print(f"模型已保存至: {final_save_dir}")
model_size = os.path.getsize(os.path.join(final_save_dir, "model.pt")) / 1024**2
print(f"模型文件大小: {model_size:.1f} MB")
print(f"\n训练摘要:")
print(f"  学生模型参数量: {param_count:,}")
print(f"  训练步数: {len(history['train_loss'])}")
if history['val_loss']:
    print(f"  最佳验证 Loss: {min(history['val_loss']):.4f}")
    print(f"  最佳验证 PPL: {min(history['val_ppl']):.2f}")
print("\n蒸馏训练完成！")